In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from matplotlib import pyplot as plt

In [2]:
Xfd_folder_path = 'skindata/train/forehead/dry'
Xnd_folder_path = 'skindata/train/nose/dry'
Xcd_folder_path = 'skindata/train/cheek/dry'
Xfo_folder_path = 'skindata/train/forehead/oily'
Xno_folder_path = 'skindata/train/nose/oily'
Xco_folder_path = 'skindata/train/cheek/oily'
yfd_folder_path = 'skindata/test/forehead/dry'
ynd_folder_path = 'skindata/test/nose/dry'
ycd_folder_path = 'skindata/test/cheek/dry'
yfo_folder_path = 'skindata/test/forehead/oily'
yno_folder_path = 'skindata/test/nose/oily'
yco_folder_path = 'skindata/test/cheek/oily'

In [3]:

# # 이미지 크기를 일괄 조정하는 함수
# def resize_and_save_images(input_folder, output_folder, target_size=(128, 128)):
#     # 폴더 내의 파일 목록을 얻습니다.
#     file_list = os.listdir(input_folder)

#     # 출력 폴더가 없다면 생성합니다.
#     os.makedirs(output_folder, exist_ok=True)

#     # 각 파일에 대해 이미지 크기를 일괄 조정합니다.
#     for file_name in file_list:
#         file_path = os.path.join(input_folder, file_name)

#         # 이미지 읽기
#         img = cv2.imread(file_path)

#         # 이미지가 올바로 읽혔는지 확인
#         if img is None:
#             print(f"Error: Unable to read the image at {file_path}")
#             continue

#         # 이미지 크기 조정
#         resized_img = cv2.resize(img, target_size)

#         # 크기가 조정된 이미지를 저장할 경로
#         save_path = os.path.join(output_folder, file_name)

#         # 크기가 조정된 이미지 저장
#         cv2.imwrite(save_path, resized_img)

# # 리사이즈할 폴더와 저장할 폴더 경로 설정
# input_train_folder = 'skindata/train'
# output_train_folder = 'resized_train'

# input_test_folder = 'skindata/test'
# output_test_folder = 'resized_test'

# # 각 얼굴 부위에 대한 훈련 데이터셋 리사이즈
# resize_and_save_images(Xfd_folder_path, os.path.join(output_train_folder, 'forehead/dry'))
# resize_and_save_images(Xnd_folder_path, os.path.join(output_train_folder, 'nose/dry'))
# resize_and_save_images(Xcd_folder_path, os.path.join(output_train_folder, 'cheek/dry'))
# resize_and_save_images(Xfo_folder_path, os.path.join(output_train_folder, 'forehead/oily'))
# resize_and_save_images(Xno_folder_path, os.path.join(output_train_folder, 'nose/oily'))
# resize_and_save_images(Xco_folder_path, os.path.join(output_train_folder, 'cheek/oily'))

# # 각 얼굴 부위에 대한 테스트 데이터셋 리사이즈
# resize_and_save_images(yfd_folder_path, os.path.join(output_test_folder, 'forehead/dry'))
# resize_and_save_images(ynd_folder_path, os.path.join(output_test_folder, 'nose/dry'))
# resize_and_save_images(ycd_folder_path, os.path.join(output_test_folder, 'cheek/dry'))
# resize_and_save_images(yfo_folder_path, os.path.join(output_test_folder, 'forehead/oily'))
# resize_and_save_images(yno_folder_path, os.path.join(output_test_folder, 'nose/oily'))
# resize_and_save_images(yco_folder_path, os.path.join(output_test_folder, 'cheek/oily'))

# # 리사이즈된 이미지를 읽어오고 확인하는 코드
# # 예시로 하나의 이미지만 확인
# sample_resized_image_path = os.path.join(output_train_folder, 'forehead/dry', os.listdir(os.path.join(output_train_folder, 'forehead/dry'))[0])
# sample_resized_image = cv2.imread(sample_resized_image_path)

# # 이미지가 올바로 읽혔는지 확인
# if sample_resized_image is not None:
#     # 이미지를 화면에 출력
#     cv2.imshow('Sample Resized Image', sample_resized_image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
# else:
#     print(f"Error: Unable to read the resized image at {sample_resized_image_path}")


In [4]:
# # 픽셀 값을 [0, 1] 범위로 스케일링하는 함수
# def preprocess_input(image):
#     return image / 255.0

# # 세그멘테이션 모델 로드 또는 훈련
# # 여기에서는 가상의 모델로 대체합니다. 실제로는 훈련된 모델을 로드하거나 적절한 세그멘테이션 모델을 사용해야 합니다.
# def load_segmentation_model():
#     # 가상의 모델을 반환합니다.
#     # 실제로는 모델을 로드하거나 훈련하는 코드로 대체해야 합니다.
#     return SomeSegmentationModel()

# class SomeSegmentationModel:
#     def predict(self, input_data):
#         # 가상의 predict 메서드
#         # 실제로는 입력 데이터에 대해 세그멘테이션을 수행하는 코드로 대체해야 합니다.
#         return np.ones_like(input_data)

# # 세그멘테이션 모델을 사용하여 이미지 세그멘테이션 수행
# segmentation_model = load_segmentation_model()  # 실제 모델을 로드하거나 훈련하는 코드로 대체

# # 데이터 경로 설정
# base_path = 'resized_skindata'
# save_path_segmented = 'segmented_skindata'

# # 얼굴 부위 및 피부 상태 설정
# face_parts = ['cheek', 'forehead', 'nose']
# skin_states = ['dry', 'oily']

# # 이미지 세그멘테이션 및 윤곽선 저장
# for state in skin_states:
#     for part in face_parts:
#         for data_split in ['resized_train', 'resized_test']:
#             folder_path = os.path.join(base_path, data_split, part, state)
#             save_path_segmented_part = os.path.join(save_path_segmented, data_split, part, state)
#             os.makedirs(save_path_segmented_part, exist_ok=True)

#             for filename in os.listdir(folder_path):
#                 img_path = os.path.join(folder_path, filename)

#                 # 이미지를 세그멘테이션하기 위해 위에서 작성한 코드 사용
#                 image_to_segment = cv2.imread(img_path)
#                 image_to_segment = cv2.cvtColor(image_to_segment, cv2.COLOR_BGR2RGB)
#                 image_to_segment = cv2.resize(image_to_segment, (128, 128))

#                 segmentation_mask = segmentation_model.predict(np.expand_dims(preprocess_input(image_to_segment), axis=0))[0]
#                 segmented_image = image_to_segment * segmentation_mask
#                 segmented_image = cv2.resize(segmented_image, (128, 128))
#                 segmented_image = preprocess_input(segmented_image)

#                 # 저장 경로 설정
#                 save_path = os.path.join(save_path_segmented_part, filename)

#                 # 이미지 저장
#                 cv2.imwrite(save_path, (segmented_image * 255).astype(np.uint8))

#                 # 윤곽선을 찾기 위한 코드
#                 dst_img = image_to_segment.copy()

#                 # 각 채널에 대해 윤곽선을 찾습니다.
#                 for i in range(3):  # 3개의 채널(BGR)에 대해 반복
#                     channel = image_to_segment[:, :, i]  # 각 채널 선택
#                     ret, otsu = cv2.threshold(channel, -1, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
#                     contours, hier = cv2.findContours(otsu, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

#                     COLOR = (0, 200, 2)
#                     cv2.drawContours(dst_img, contours, -1, COLOR, 2)

#                 # 윤곽선을 저장할 경로 설정
#                 save_path_contours = os.path.join(save_path_segmented_part, 'contours', filename)
#                 os.makedirs(os.path.dirname(save_path_contours), exist_ok=True)

#                 # 윤곽선 저장
#                 cv2.imwrite(save_path_contours, dst_img)


In [5]:
# 데이터 경로 설정
base_path = 'segmented_skindata'

# 얼굴 부위 및 피부 상태 설정
face_parts = ['forehead', 'nose', 'cheek']
skin_states = ['dry', 'oily']

# 데이터 불러오기
data = []
labels = []

for state in skin_states:
    for part in face_parts:
        folder_path = os.path.join(base_path, 'resized_train', part, state)

        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)

            # Additional check to skip empty images
            if os.path.isfile(img_path):
                img = cv2.imread(img_path)

                if img is not None:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (128, 128))

                    data.append(img)
                    labels.append(f'{part}_{state}') 
                else:
                    print(f"Warning: Unable to read the image at {img_path}")


# 데이터와 레이블을 numpy 배열로 변환
data = np.array(data)
labels = np.array(labels)

# 레이블 인코딩
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
encoded_labels = to_categorical(encoded_labels)

# 데이터 분할 (8:2 비율)
X_train, X_test, y_train, y_test = train_test_split(data, encoded_labels, test_size=0.2, random_state=42)


In [6]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(128,128, 3))

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
18/18 [==============================] - 79s 4s/step - loss: 1.2595 - accuracy: 0.4740 - val_loss: 1.1108 - val_accuracy: 0.5000
Epoch 2/10
18/18 [==============================] - 63s 4s/step - loss: 0.4465 - accuracy: 0.8368 - val_loss: 0.8358 - val_accuracy: 0.6319
Epoch 3/10
18/18 [==============================] - 55s 3s/step - loss: 0.1995 - accuracy: 0.9358 - val_loss: 1.2202 - val_accuracy: 0.6250
Epoch 4/10
18/18 [==============================] - 72s 4s/step - loss: 0.1561 - accuracy: 0.9323 - val_loss: 1.7162 - val_accuracy: 0.6319
Epoch 5/10
18/18 [==============================] - 73s 4s/step - loss: 0.1360 - accuracy: 0.9566 - val_loss: 2.1080 - val_accuracy: 0.5417
Epoch 6/10
18/18 [==============================] - 68s 4s/step - loss: 0.1717 - accuracy: 0.9410 - val_loss: 2.4735 - val_accuracy: 0.5278
Epoch 7/10
18/18 [==============================] - 74s 4s/step - loss: 0.1990 - accuracy: 0.9340 - val_loss: 2.3915 - val_accuracy: 0.5972
Epoch 8/10
18/18 [==

In [7]:
# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

# 테스트 데이터에 대한 예측
predictions = model.predict(X_test)

# 예측 결과 출력
for i in range(5):  # 예측 결과 중에서 처음 5개만 출력
    predicted_class = label_encoder.classes_[np.argmax(predictions[i])]
    actual_class = label_encoder.classes_[np.argmax(y_test[i])]
    print(f'Actual: {actual_class}, Predicted: {predicted_class}')

5/5 [==============================] - 4s 756ms/step - loss: 2.0480 - accuracy: 0.5833
Test Loss: 2.0479650497436523, Test Accuracy: 0.5833333134651184
Actual: cheek_dry, Predicted: cheek_dry
Actual: cheek_dry, Predicted: cheek_dry
Actual: forehead_dry, Predicted: forehead_dry
Actual: nose_dry, Predicted: cheek_oily
Actual: forehead_oily, Predicted: forehead_oily
